In [44]:
import warnings
import sys
sys.path.append('../')
from Notebooks.helper import load_env

warnings.filterwarnings('ignore')
load_env()

Import all the required dependencies

In [45]:
from haystack import Pipeline
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
import os
import pandas as pd
import json

In [46]:
prompt = """
Answer the question below:
Question: {{ query }}
"""
prompt_builder = PromptBuilder(template=prompt)
generator = OpenAIGenerator(model="gpt-4")

query_pipeline = Pipeline()
query_pipeline.add_component("prompt", prompt_builder)
query_pipeline.add_component("generator", generator)

query_pipeline.connect("prompt", "generator")

🚅 Components
  - prompt: PromptBuilder
  - generator: OpenAIGenerator
🛤️ Connections
  - prompt.prompt -> generator.prompt (str)

In [47]:
df = pd.read_csv('gics-map-2018.csv')
for index, row in df.iterrows():
    # Access the values of each column in the current row
    sub_industry_description = row['SubIndustryDescription']
    question = f"Give me an unformatted list of Climate related risks and mitigations for a company who's industry is classified as {sub_industry_description}? Provide response in a single JSON array of objects, each object has an attribute risk and an attribute mitigation"
    result = query_pipeline.run(
        {
            "prompt": {"query": question},
        }
    )
    print('Risks - '+result['generator']['replies'][0])
    df.at[row.name,'Climate Risk Mitigation'] = result['generator']['replies'][0]

    # question = f"Give me an unformatted list of Risk mitigations for the following Risks: {result['generator']['replies'][0]}? Provide response in this exact JSON format: [\"mitigation\",\"mitigation\"]"
    # result = query_pipeline.run(
    #     {
    #         "prompt": {"query": question},
    #     }
    # )
    print('Mitigations - '+result['generator']['replies'][0])

    
    df.to_csv('Gics_modified_gpt4.csv')

SyntaxError: unterminated string literal (detected at line 5) (3034468201.py, line 5)

Set up the document ingestion pipeline

In [ ]:
print(result["generator"]["replies"][0])

{
    "industries": ["Bicycle industry", "Retail industry"]
}


In [ ]:
industry_analysis_pipeline = Pipeline()